In [1]:
import requests
from bs4 import BeautifulSoup
import random
import re
import time
from pprint import pprint
import json
import pandas as pd

Функция для получения данных

In [2]:
def get_data(url, params = '', headers ='', proxy = ''): 
        r = requests.get(url, params, headers=headers, proxies=proxy)
        return r.text
    

Функция для выбора случайного юзер-агента

In [3]:
def choose_user_agent(user_agents):
        user_agent = user_agents[random.randrange(len(user_agents))]
        headers = {'user-agent': user_agent}
        return headers

Чтение данных юзер-агентов

In [4]:
with open(r'C:\Users\Lenovo\Desktop\diplom\desktop_user_agent.txt', 'r', encoding='utf-8') as fp:
    user_agents = [line.strip() for line in fp.read().split('\n')]

In [5]:
#url = f'https://career.habr.com/vacancies?page={page_number}&q=аналитик&type=all'
#data = get_data(url = url, headers = choose_user_agent(user_agents))

id собранных ранее вакансий

In [6]:
with open(r'C:\Users\Lenovo\Desktop\diplom\habr_data.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)
all_ids = set()
for vac in all_vacs:
    all_ids.add(vac['id'])
len(all_ids)

405

Получение данных о вакансиях и отбор только тех, что нет в общем файле

In [7]:
vac_info = []
page_number = 1
while True:
    #print(f'{page_number} started')
    url = f'https://career.habr.com/vacancies?locations[]=ct_444&page={page_number}&q=аналитик&type=all'
    data = get_data(url = url, headers = choose_user_agent(user_agents))
    soup = BeautifulSoup(data, 'lxml')
    blocks = soup.find_all('a', attrs = {'class': 'vacancy-card__title-link'})
    if len(blocks) == 0:
        break
    for block in blocks:
        vac_id = block['href']
        if int(vac_id[11:]) in all_ids:
            continue
        if re.search('аналитик|BI|analyst', block.text.lower()) == None:
            continue
        #vac_id = '/vacancies/1000115845'
        vac_url = f'https://career.habr.com{vac_id}'
        vac_data = get_data(vac_url, headers = choose_user_agent(user_agents))
        area = ''
        description = ''
        employment = ''
        experience = ''
        id = ''
        key_skills = [] 
        name = ''
        publication_date = ''
        salary = ''
        schedule = ''
        specializations = ''
        proffesional_roles= ''
        employer = ''
        vac_soup = BeautifulSoup(vac_data, 'lxml').find('div', attrs = {'class': 'vacancy-description__text'})
        try: 
            description = vac_soup.get_text(' ')
        except:
            continue
        vac_soup = BeautifulSoup(vac_data, 'lxml').find_all('script', attrs = {'type': 'application/json'})[0].text
        vac_json = json.loads(vac_soup)
        name = vac_json['vacancy']['title']
        id = vac_json['vacancy']['id']
        salary = vac_json['vacancy']['salary']
        for skill in vac_json['vacancy']['skills']:
            key_skills.append(skill['title'])
        try: 
            experience = vac_json['vacancy']['salaryQualification']['title']
        except:
            experience = ''
        employment = vac_json['vacancy']['employmentType']
        employer = vac_json['vacancy']['company']['title']
        publication_date = vac_json['vacancy']['publishedDate']['date']
        area = vac_json['vacancy']['shortGeo']
        if vac_json['vacancy']['remoteWork'] == 'True':
            schedule = 'Можно удаленно'
        vac_info.append({
            'area': area,
            'description': description,
            'employment': employment,
            'experience': experience,
            'id': id,
            'key_skills': key_skills,
            'name': name,
            'publication_date': publication_date,
            'salary': salary,
            'schedule': schedule,
            'specializations': specializations,
            'proffesional_roles': proffesional_roles,
            'employer': employer,
            'source': 'career.habr.com'}
        )
    print(f'{page_number} finished')
    page_number += 1
    

1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
10 finished
11 finished
12 finished
13 finished
14 finished
15 finished
16 finished
17 finished
18 finished
19 finished
20 finished
21 finished
22 finished
23 finished
24 finished


Функция для записи полученных за день данных в файл

In [8]:
def add_data_to_file(vac_info):
    with open(r'C:\Users\Lenovo\Desktop\diplom\habr_data.json', 'r', encoding='utf-8') as fp:
        all_vacs = json.load(fp)
    all_vacs = all_vacs + vac_info
    with open(r'C:\Users\Lenovo\Desktop\diplom\habr_data.json', 'w', encoding='utf-8') as fp:
        json.dump(all_vacs, fp, ensure_ascii=False)

Сохранения данных в excel

In [9]:
def from_json_to_excel():
    with open(r'C:\Users\Lenovo\Desktop\diplom\habr_data.json', 'r', encoding='utf-8') as fp:
        to_excel = json.load(fp)
    df = pd.DataFrame(to_excel)
    df.to_excel(r'C:\Users\Lenovo\Desktop\diplom\habr_data.xlsx')

In [10]:
add_data_to_file(vac_info)

In [11]:
#from_json_to_excel()